In [ ]:
import pandas as pd
import numpy as np


import torch
from sklearn.utils.class_weight import compute_class_weight
import torch.nn as nn

from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score, confusion_matrix, classification_report

from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
from transformers import AutoModel
from safetensors.torch import load_model


## 8. Classification: predict movie success

### 8.1. Data load and preprocessing

In [152]:
imdb_data = pd.read_csv("imdb_top_1000.csv")

In [153]:
def preprocess_imdb_data(imdb_data):
    imdb_data["Gross"] = imdb_data["Gross"].apply(lambda x: float(x.replace(",", "")) if isinstance(x, str) else x)
    imdb_data["Released_Year"] = pd.to_datetime(imdb_data["Released_Year"], format="%Y", errors="coerce")
    imdb_data["Runtime"] = imdb_data["Runtime"].apply(lambda x: int(x.split()[0]))
    imdb_data["Genre"] = imdb_data["Genre"].apply(lambda x: x.split(", "))
    return imdb_data

In [154]:
imdb_data = preprocess_imdb_data(imdb_data)

### 8.2. Transform for classification model

Movies with a rating greater than 8 will be considered a hit.

In [155]:
transformer_model = SentenceTransformer('all-MiniLM-L6-v2')

In [156]:
def encode_multilabel_column(dataframe, column_name):
    encoder = MultiLabelBinarizer()
    encoded_column = encoder.fit_transform(dataframe[column_name])
    encoded_df = pd.DataFrame(encoded_column, columns=encoder.classes_, index=dataframe.index)
    return encoded_df


def encode_onehot_column(dataframe, column_name):
    encoder = OneHotEncoder(sparse_output=False)
    encoded_column = encoder.fit_transform(dataframe[[column_name]])

    encoded_df = pd.DataFrame(encoded_column, columns=encoder.get_feature_names_out([column_name]), index=dataframe.index)
    return encoded_df


def process_dataset(dataset):
    dataset["Overview_Embedding"] = dataset["Overview"].apply(lambda x: transformer_model.encode(x))
    embeddings = np.vstack(dataset["Overview_Embedding"].to_numpy())
    embeddings_df = pd.DataFrame(embeddings, columns=[f'emb_{i}' for i in range(embeddings.shape[1])])

    feat_data = dataset[["Genre", "Runtime", "Gross" ,"Certificate", "No_of_Votes"]]

    # NaN values will be treated as a separate category
    feat_data["Certificate"] = feat_data["Certificate"].apply(lambda x: "Unknown" if pd.isna(x) else x)

    genre_df = encode_multilabel_column(feat_data, "Genre")
    cert_df = encode_onehot_column(feat_data, "Certificate")

    final_data = pd.concat(
        [
            embeddings_df,
            genre_df,
            cert_df,
            feat_data[["No_of_Votes", "Gross", "Runtime"]],
        ],
        axis=1,
    )
    

    # Success definition: IMDB_Rating > 8.0
    labels = np.array([1 if x > 8.2 else 0 for x in dataset["IMDB_Rating"].to_numpy()])
    return final_data, labels

In [ ]:
data, labels = process_dataset(imdb_data)

C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\3177057602.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feat_data["Certificate"] = feat_data["Certificate"].apply(lambda x: "Unknown" if pd.isna(x) else x)


128

In [8]:
data_mod = data.drop(columns=["Gross"])
data_train, data_test, labels_train, labels_test = train_test_split(data_mod, labels, test_size=0.2, random_state=32)

In [9]:
print(len(labels_train))
print(np.sum(labels_train))
print(len(labels_test))
print(np.sum(labels_test))

800
97
200
31


In [10]:
weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
weights

#class_weights = torch.tensor(weights, dtype=torch.float)
#print(class_weights)


array([0.5733945, 3.90625  ])

### 8.3. Model comparison

#### 8.3.1. Logistic Regression

In [ ]:
#define model
clf = LogisticRegression(random_state=0, class_weight="balanced", max_iter=1000).fit(data_train, labels_train)

C:\Users\eetu\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#predictions
preds = clf.predict(data_test)

In [16]:
f1 = f1_score(labels_test, preds)
cm = confusion_matrix(labels_test, preds)
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)

print(classification_report(labels_test, preds, digits=4))

F1-score:  0.4944

Confusion Matrix:
 [[133  36]
 [  9  22]]
              precision    recall  f1-score   support

           0     0.9366    0.7870    0.8553       169
           1     0.3793    0.7097    0.4944        31

    accuracy                         0.7750       200
   macro avg     0.6580    0.7483    0.6748       200
weighted avg     0.8502    0.7750    0.7994       200



#### 8.3.2. SVC

In [17]:
svc_model = SVC(
    C=0.5,
    kernel='rbf',
    gamma='scale',
    class_weight="balanced",
    max_iter=1000
    #random_state=42
)

svc_model.fit(data_train, labels_train)

,C,0.5
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,'balanced'
,verbose,False


In [18]:
y_pred = svc_model.predict(data_test)

rmse = np.sqrt(mean_squared_error(labels_test, y_pred))
mae = mean_absolute_error(labels_test, y_pred)

In [19]:
f1 = f1_score(labels_test, y_pred)
cm = confusion_matrix(labels_test, y_pred)
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)

print(classification_report(labels_test, y_pred, digits=4))

F1-score:  0.5128

Confusion Matrix:
 [[142  27]
 [ 11  20]]
              precision    recall  f1-score   support

           0     0.9281    0.8402    0.8820       169
           1     0.4255    0.6452    0.5128        31

    accuracy                         0.8100       200
   macro avg     0.6768    0.7427    0.6974       200
weighted avg     0.8502    0.8100    0.8248       200



#### 8.3.3. Random Forest

In [ ]:
rfc_model = BalancedRandomForestClassifier(
    n_estimators=200,
    #class_weight="balanced"
    #random_state=42
)

rfc_model.fit(data_train, labels_train)

,n_estimators,200
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,False
,oob_score,False


In [24]:
y_pred = rfc_model.predict(data_test)

rmse = np.sqrt(mean_squared_error(labels_test, y_pred))
mae = mean_absolute_error(labels_test, y_pred)

In [25]:
f1 = f1_score(labels_test, y_pred)
cm = confusion_matrix(labels_test, y_pred)
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)

print(classification_report(labels_test, y_pred, digits=4))

F1-score:  0.4091

Confusion Matrix:
 [[165   4]
 [ 22   9]]
              precision    recall  f1-score   support

           0     0.8824    0.9763    0.9270       169
           1     0.6923    0.2903    0.4091        31

    accuracy                         0.8700       200
   macro avg     0.7873    0.6333    0.6680       200
weighted avg     0.8529    0.8700    0.8467       200



#### 8.3.4. BERT based text classifier

In [129]:
model_path = "google-bert/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)

model = AutoModelForSequenceClassification.from_pretrained(model_path,  num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [130]:
bert_data = imdb_data["Overview"]

# split data for train, test and validation sets

data_train_bert, data_test_bert, labels_train_bert, labels_test_bert = train_test_split(bert_data, labels, test_size=0.2, random_state=8)
data_train_bert, data_valid_bert, labels_train_bert, labels_valid_bert = train_test_split(data_train_bert, labels_train_bert, test_size=0.2, random_state=5)

In [131]:
print(np.sum(labels_train_bert))
print(np.sum(labels_valid_bert))
print(np.sum(labels_test_bert))

80
22
26


In [132]:
train_dataset = Dataset.from_dict({
    "overview": data_train_bert.tolist(),
    "label": labels_train_bert.tolist()
})

eval_dataset = Dataset.from_dict({
    "overview": data_valid_bert.tolist(),
    "label": labels_valid_bert.tolist()
})

In [133]:
def tokenize_dataset(dataset):
    return tokenizer(
        dataset["overview"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

tokenized_train = train_dataset.map(tokenize_dataset, batched=True)
tokenized_eval = eval_dataset.map(tokenize_dataset, batched=True)

Map: 100%|██████████| 160/160 [00:00<00:00, 11414.43 examples/s]


In [134]:
# freeze all base model parameters
for name, param in model.base_model.named_parameters():
    param.requires_grad = False
    
for name, param in model.base_model.named_parameters():
    if "pooler" in name:
        param.requires_grad = True

In [135]:
# metrics to show during training loop
accuracy = evaluate.load("accuracy")
auc_score = evaluate.load("roc_auc")

In [136]:
#Define new trainer with weighted loss
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        
        loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

In [137]:
# metrics during training 
def compute_metrics(eval_pred):
    # get predictions
    predictions, labels = eval_pred
    
    # apply softmax to get probabilities
    probabilities = np.exp(predictions) / np.exp(predictions).sum(-1, keepdims=True)
    # use probabilities of the positive class for ROC AUC
    positive_class_probs = probabilities[:, 1]
    # compute auc
    auc = np.round(auc_score.compute(prediction_scores=positive_class_probs,  references=labels)['roc_auc'],3)
    
    # predict most probable class
    predicted_classes = np.argmax(predictions, axis=1)
    # compute accuracy
    acc = np.round(accuracy.compute(predictions=predicted_classes,  references=labels)['accuracy'],3)
    
    return {"Accuracy": acc, "AUC": auc}

In [138]:
# hyperparameters
lr = 7e-4
batch_size = 8
num_epochs = 20

In [ ]:
training_args = TrainingArguments(
    output_dir="./bert-movie_classifier",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
)

In [140]:
# create trainer and train
weighted_trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

class_weights = torch.tensor(weights, dtype=torch.float)

weighted_trainer.class_weights = class_weights
weighted_trainer.train()

C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\494162676.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedLossTrainer.__init__`. Use `processing_class` instead.
  weighted_trainer = WeightedLossTrainer(
                                                 
  5%|▌         | 80/1600 [00:03<00:57, 26.25it/s]  

{'loss': 0.8728, 'grad_norm': 18.394929885864258, 'learning_rate': 0.000665, 'epoch': 1.0}










                                                 
                                                
  5%|▌         | 80/1600 [00:03<00:57, 26.25it/s]


{'eval_loss': 0.8300922513008118, 'eval_Accuracy': 0.181, 'eval_AUC': 0.545, 'eval_runtime': 0.7124, 'eval_samples_per_second': 224.585, 'eval_steps_per_second': 28.073, 'epoch': 1.0}


                                                  
 10%|█         | 160/1600 [00:06<00:54, 26.56it/s] 

...







                                                   
                                                

100%|██████████| 1600/1600 [01:19<00:00, 26.59it/s]



{'eval_loss': 0.828974723815918, 'eval_Accuracy': 0.681, 'eval_AUC': 0.602, 'eval_runtime': 0.7131, 'eval_samples_per_second': 224.357, 'eval_steps_per_second': 28.045, 'epoch': 20.0}


                                                   
100%|██████████| 1600/1600 [01:20<00:00, 19.96it/s]

{'train_runtime': 80.1447, 'train_samples_per_second': 159.711, 'train_steps_per_second': 19.964, 'train_loss': 0.6451917147636413, 'epoch': 20.0}


TrainOutput(global_step=1600, training_loss=0.6451917147636413, metrics={'train_runtime': 80.1447, 'train_samples_per_second': 159.711, 'train_steps_per_second': 19.964, 'total_flos': 1683910754304000.0, 'train_loss': 0.6451917147636413, 'epoch': 20.0})

In [141]:
#load best epoch for testing
checkpoint_path = "bert-movie_classifier\checkpoint-800"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\3762372812.py:2: SyntaxWarning: invalid escape sequence '\c'
  checkpoint_path = "bert-movie_classifier\checkpoint-800"


In [142]:
# get predictions
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

test_dataset = Dataset.from_dict({
    "overview": data_test_bert.tolist(),
    "label": labels_test_bert.tolist()
})

tokenized_test = test_dataset.map(tokenize_dataset, batched=True)

predictions = trainer.predict(tokenized_test)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids
print(y_pred)

C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\1565688795.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
100%|██████████| 25/25 [00:00<00:00, 30.12it/s]

[0 0 0 0 1 0 1 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0]


In [143]:
# calculate Accuracy, F1, percision, recall and confusion matrix
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)

print(classification_report(y_true, y_pred, digits=4))

F1-score:  0.2462

Confusion Matrix:
 [[143  31]
 [ 18   8]]
              precision    recall  f1-score   support

           0     0.8882    0.8218    0.8537       174
           1     0.2051    0.3077    0.2462        26

    accuracy                         0.7550       200
   macro avg     0.5467    0.5648    0.5499       200
weighted avg     0.7994    0.7550    0.7747       200



#### 8.3.5. BERT based multimodal classifier 

In [ ]:
# CUSTOM MODEL FOR MULTIMODAL CLASSIFICATION WITH BERT AND LINEAR LAYER FOR METADATA (CERTIFICATE, RUNTIME, NUMBER OF VOTES)

class BERTWithMetadata(nn.Module):
    def __init__(self, model_name, num_metadata_features, num_labels=2,  class_weights=None):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.2)
        
        # Linear layer for metadata
        self.metadata_fc = nn.Linear(num_metadata_features, 128)
        
        # Combine [CLS] + metadata
        self.classifier = nn.Linear(self.bert.config.hidden_size + 128, num_labels)
        
        if class_weights is not None:
            self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))
        else:
            self.class_weights = None
            
    
    def forward(self, input_ids, attention_mask, metadata, labels=None, **kwargs):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        
        meta_output = torch.relu(self.metadata_fc(metadata))
        combined = torch.cat((pooled_output, meta_output), dim=1)
        combined = self.dropout(combined)
        logits = self.classifier(combined)
        
        loss = None
        if labels is not None:
            if self.class_weights is not None:
                loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            else:
                loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
        return {"loss": loss, "logits": logits} if loss is not None else {"logits": logits}

In [ ]:
#split data into train, test, validation sets

encoder = OneHotEncoder(sparse_output=False)

bert_data_improved = imdb_data[["Overview", "Certificate", "Runtime", "No_of_Votes"]]
bert_data_improved["label"] = labels

train_df, test_df = train_test_split(bert_data_improved, test_size=0.2, random_state=7, stratify=bert_data_improved["label"])
train_df, eval_df = train_test_split(train_df, test_size=0.2, random_state=7, stratify=train_df["label"])




C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\453121321.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bert_data_improved["label"] = labels


In [ ]:
# create metadata for model
train_encodings = tokenizer(
    train_df["Overview"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

eval_encodings = tokenizer(
    eval_df["Overview"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

test_encodings = tokenizer(
    test_df["Overview"].tolist(),
    truncation=True,
    padding=True,
    max_length=256
)

certificate_train = train_df[["Certificate"]].fillna("Unknown")
certificate_eval  = eval_df[["Certificate"]].fillna("Unknown")
certificate_test  = test_df[["Certificate"]].fillna("Unknown")

ohe = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
certificate_train = ohe.fit_transform(certificate_train)
certificate_eval  = ohe.transform(certificate_eval)
certificate_test  = ohe.transform(certificate_test)

num_features_train = train_df[["Runtime", "No_of_Votes"]].fillna(0)
num_features_eval  = eval_df[["Runtime", "No_of_Votes"]].fillna(0)
num_features_test  = test_df[["Runtime", "No_of_Votes"]].fillna(0)

scaler = StandardScaler()
num_train = scaler.fit_transform(num_features_train)
num_eval  = scaler.transform(num_features_eval)
num_test  = scaler.transform(num_features_test)

# Combine
train_metadata = np.concatenate([certificate_train, num_train], axis=1)
eval_metadata  = np.concatenate([certificate_eval, num_eval], axis=1)
test_metadata  = np.concatenate([certificate_test, num_test], axis=1)

In [161]:
# get labels
train_labels = train_df["label"].values
eval_labels  = eval_df["label"].values
test_labels  = test_df["label"].values

In [162]:
# custom dataset to use with model

class MovieDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, metadata, labels):
        self.encodings = encodings
        self.metadata = metadata
        self.labels = labels

    def __getitem__(self, idx):
        
        if isinstance(idx, list) or isinstance(idx, np.ndarray):
            batch = [self.__getitem__(i) for i in idx]
            return {key: torch.stack([d[key] for d in batch]) for key in batch[0]}

        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["metadata"] = torch.tensor(self.metadata[idx], dtype=torch.float)
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

In [163]:
train_dataset = MovieDataset(train_encodings, train_metadata, train_labels)
eval_dataset  = MovieDataset(eval_encodings,  eval_metadata,  eval_labels)
test_dataset  = MovieDataset(test_encodings,  test_metadata,  test_labels)

In [164]:
# custom data collator
def collate_fn(batch):
    
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    metadata = [item["metadata"] for item in batch]
    labels = [item["labels"] for item in batch]

    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
    attention_mask = torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True, padding_value=0)
    metadata = torch.stack(metadata)
    labels = torch.stack(labels)

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "metadata": metadata,
        "labels": labels
    }

In [ ]:
# initiate the model using custom class
model = BERTWithMetadata("bert-base-uncased", num_metadata_features=train_metadata.shape[1], num_labels=2, class_weights=class_weights)


C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\1628355477.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))


In [ ]:
#TRAIN MODEL

training_args = TrainingArguments(
    output_dir="./bert-movie_classifier_with_metadata",
    learning_rate=5e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    remove_unused_columns=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)


trainer.train()

C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\1282081318.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  3%|▎         | 80/2400 [00:04<02:08, 18.06it/s]

{'loss': 0.7571, 'grad_norm': 5.260904312133789, 'learning_rate': 0.00048333333333333334, 'epoch': 1.0}


                                                 
  3%|▎         | 80/2400 [00:04<02:08, 18.06it/s]

{'eval_loss': 0.6345117688179016, 'eval_Accuracy': 0.55, 'eval_AUC': 0.844, 'eval_runtime': 0.2469, 'eval_samples_per_second': 647.938, 'eval_steps_per_second': 80.992, 'epoch': 1.0}


  7%|▋         | 160/2400 [00:10<02:02, 18.35it/s]

...

                                                   
100%|██████████| 2400/2400 [02:49<00:00, 17.61it/s]

{'eval_loss': 0.5653203725814819, 'eval_Accuracy': 0.875, 'eval_AUC': 0.823, 'eval_runtime': 0.2425, 'eval_samples_per_second': 659.688, 'eval_steps_per_second': 82.461, 'epoch': 30.0}


100%|██████████| 2400/2400 [02:51<00:00, 14.01it/s]

{'train_runtime': 171.319, 'train_samples_per_second': 112.072, 'train_steps_per_second': 14.009, 'train_loss': 0.6177941783269246, 'epoch': 30.0}


TrainOutput(global_step=2400, training_loss=0.6177941783269246, metrics={'train_runtime': 171.319, 'train_samples_per_second': 112.072, 'train_steps_per_second': 14.009, 'total_flos': 0.0, 'train_loss': 0.6177941783269246, 'epoch': 30.0})

In [169]:
#load best epoch for evaluation
checkpoint_path = "bert-movie_classifier_with_metadata\checkpoint-560\model.safetensors"

model = BERTWithMetadata("bert-base-uncased", num_metadata_features=eval_metadata.shape[1], num_labels=2, class_weights=class_weights)
load_model(model, checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained("bert-movie_classifier_with_metadata\checkpoint-560")
model.eval()

<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
<>:2: SyntaxWarning: invalid escape sequence '\c'
<>:6: SyntaxWarning: invalid escape sequence '\c'
C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\3883105923.py:2: SyntaxWarning: invalid escape sequence '\c'
  checkpoint_path = "bert-movie_classifier_with_metadata\checkpoint-560\model.safetensors"
C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\3883105923.py:6: SyntaxWarning: invalid escape sequence '\c'
  tokenizer = AutoTokenizer.from_pretrained("bert-movie_classifier_with_metadata\checkpoint-560")
C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\1628355477.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("class_weights", torch.tensor(class_weights, dtype=torch.float))


BERTWithMetadata(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [170]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\eetu\AppData\Local\Temp\ipykernel_40868\2378322192.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [171]:
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids
print(y_pred)

100%|██████████| 25/25 [00:00<00:00, 52.53it/s]

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0]


In [172]:
f1 = f1_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
print(f"F1-score:  {f1:.4f}")
print("\nConfusion Matrix:\n", cm)

print(classification_report(y_true, y_pred, digits=4))

F1-score:  0.4478

Confusion Matrix:
 [[148  26]
 [ 11  15]]
              precision    recall  f1-score   support

           0     0.9308    0.8506    0.8889       174
           1     0.3659    0.5769    0.4478        26

    accuracy                         0.8150       200
   macro avg     0.6483    0.7137    0.6683       200
weighted avg     0.8574    0.8150    0.8315       200

